In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.abspath('')
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import re

import importlib
import utils
importlib.reload(utils)
from tqdm import tqdm
import pickle

from utils import get_parsed, get_parsed_remove_comment, baseball_base_url

In [3]:
content = get_parsed_remove_comment(baseball_base_url + '/teams/')

In [5]:
teams_active = content.find('table', id='teams_active')
all_rows = teams_active.find_all('tr')

## Columns Information

In [7]:
ths = all_rows[0].find_all('th')
column_names = [th.text for th in ths]
column_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else '' for th in ths]
column_exp_tip = [th.attrs['data-tip'] if 'data-tip' in th.attrs else '' for th in ths]

In [8]:
df_columns_exp = pd.DataFrame([column_exp_stat, column_exp_tip], columns=column_names)

In [9]:
df_columns_exp

,Rk,Franchise,From,To,G,W,L,W-L%,G>.500,Divs,...,Playoffs,Players,HOF#,R,AB,H,HR,BA,RA,ERA
0,ranker,franchise_name,year_min,year_max,G,W,L,win_loss_perc,g_over_500,divisions,...,playoffs,players_number,HOFers,R,AB,H,HR,batting_avg,RA,earned_run_avg
1,<strong>Rank</strong><br>This is a count of th...,,First Year,Last Year,Games Played or Pitched,Wins,Losses,<strong>Win-Loss Percentage</strong><br>W / (W...,<strong>Games over .500</strong><br>If positiv...,<strong>Divisions Won</strong><br>The number o...,...,<strong>Playoffs</strong><br>This shows either...,<strong>Number of Players</strong>,<strong>Number of Former Players who are in Ha...,Runs Scored/Allowed,At Bats,Hits/Hits Allowed,Home Runs Hit/Allowed,<strong>Hits/At Bats</strong><br>For recent ye...,Runs Allowed,<strong>9 * ER / IP</strong><br>For recent yea...


## Scraping 

In [14]:
data = []

for row in all_rows[1:]:
    if 'class' not in row.attrs:
        rk = row.find('th').text
        all_tds =  row.find_all('td')
        rest = [td.text for td in all_tds]
        url = all_tds[0].a['href']
        data.append([rk] + rest + [url])
df_all_teams = pd.DataFrame(data, columns=column_names + ['Franchise_url'])

In [15]:
df_all_teams.head()

,Rk,Franchise,From,To,G,W,L,W-L%,G>.500,Divs,...,Players,HOF#,R,AB,H,HR,BA,RA,ERA,Franchise_url
0,1,Arizona Diamondbacks,1998,2020,"3,624","1,788","1,836",.493,-48,5,...,511,2,"16,492","123,973","31,804","3,851",.257,"16,608",4.22,/teams/ARI/
1,2,Atlanta Braves,1876,2020,"21,570","10,732","10,684",.501,48,20,...,2033,53,"96,095","738,734","192,010","14,003",.260,"95,224",3.67,/teams/ATL/
2,3,Baltimore Orioles,1901,2020,"18,666","8,793","9,763",.474,-970,9,...,1873,34,"80,495","634,197","163,983","13,595",.259,"85,779",4.04,/teams/BAL/
3,4,Boston Red Sox,1901,2020,"18,653","9,626","8,944",.518,682,10,...,1838,37,"87,054","638,406","170,591","13,793",.267,"83,499",3.91,/teams/BOS/
4,5,Chicago Cubs,1876,2020,"21,607","11,016","10,430",.514,586,8,...,2117,45,"99,513","739,895","194,194","14,449",.262,"95,943",3.68,/teams/CHC/


In [184]:
folder = 'teams'
file_name = 'all_teams' + '.pkl'
path = os.path.join(folder, file_name)
with open(path, 'wb') as f:
    pickle.dump(df_all_teams, f)

## Scrape each team data

past franchise name

Retired Numbers

All-Time Top Players

Franchise History

all uniform numbers by number and year

detail_season_by_season_data

    game data
    top player
    Team Batting
    Team Pitching
    Current 40-Man Roster (only for current season)
    Full-Season Roster & Games by Position
    Coaching Staff
    Team Fielding--Totals
    Team Player Value--Batters
    Team Player Value--Pitchers
    

In [206]:
def generic_table_analyze(table):
    '''
    this table has to have thead and tbody
    thead is one row of tr, tr is composed of all th, th has data-stat and data-tip 
        attributes to record the meaning of each column
    tbody is multiple rows of tr, 
        each tr is th + [td]
    if this format is not followed, can't return result
    returning only df and exp
    '''
    head = table.thead
    columns_name = [th.text for th in head.find_all('th')]
    column_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else "" for th in head.find_all('th')]
    column_exp_tip = [th.attrs['data-tip'] if 'data-tip' in th.attrs else "" for th in head.find_all('th')]
    df_exp = pd.DataFrame([column_exp_stat, column_exp_tip], columns=columns_name)
    
    data = []
    body = table.tbody
    for th in body.find_all('tr'):
        first_column = th.find('th').text
        rest = [td.text for td in th.find_all('td')]
        data.append([first_column] + rest)
    
    df = pd.DataFrame(data, columns=columns_name)
    
    return {'df': df, 'exp': df_exp}

In [207]:
def info(content):
    info_div = content.find('div', id='info')
    small = {}
    for p in info_div.find_all('p'):
        piece = p.text
        if ":" in piece:
            key, val = piece.split(':')
            key = key.strip()
            val = val.strip()
            small[key] = val
    return {'dict': small}, small

def all_history(content):
    '''
    history names
    '''
    hist = content.find('div', id='all_history')
    
    table = hist.find('table')
    head = table.thead
    columns_name = [th.text for th in head.find_all('th')]
    column_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else "" for th in head.find_all('th')]
    column_exp_tip = [th.attrs['data-tip'] if 'data-tip' in th.attrs else "" for th in head.find_all('th')]
    df_exp = pd.DataFrame([column_exp_stat, column_exp_tip], columns=columns_name)
    
    data = []
    body = table.tbody
    for th in body.find_all('tr'):
        name = th.find('th').text
        rest = [td.text for td in th.find_all('td')]
        data.append([name] + rest)
    
    foot = table.tfoot
    for th in foot.find_all('tr'):
        name = th.find('th').text
        rest = [td.text for td in th.find_all('td')]
        data.append([name] + rest)
    
    df = pd.DataFrame(data, columns=columns_name)
    
    return {'df': df, 'exp': df_exp}

def retired(content):
    retire = content.find('div', {"id" : re.compile('all_teams_retired_unis*')})
    hrefs = retire.find_all('a')
    columns = ['Name', 'Number']
    data = []
    for href in hrefs:
        #print(href)
        if 'data-tip' in href.attrs:
            name = href.attrs['data-tip']
            number = href.svg.text
            data.append([name, number])
    
    df = pd.DataFrame(data, columns=columns)
    return {'df': df}

def all_time_top(content):
    all_time = content.find('div', id='div_teams_war_images')
    hrefs = all_time.find_all('img')
    columns = ['Rank', 'Name', 'Stat', 'Image_src']
    data = []
    for href in hrefs:
        if 'data-tip' in href.attrs:
            name = ' '.join(href.attrs['alt'].split(" ")[2:])
            src = href['src']
            rank = href.attrs['data-tip'].split('.')[0]
            stat = re.search('\(.*\)', href.attrs['data-tip']).group(0)
            data.append([rank, name, stat, src])
    
    df = pd.DataFrame(data, columns=columns)
    return {'df': df}

def each_year(content):
    '''
    each year is a dict, representing all statistics for that year
    '''
    # time line results
    time_line = content.find('div', id='timeline_results')
    time_line_result = [li.text for li in time_line.find_all('li')]
    # top players for this season
    top = all_time_top(content)
    # Team Batting
    table_batting = content.find('table', id='team_batting')
    batting_result = generic_table_analyze(table_batting)
    # Team Pitching
    table_pitching = content.find('table', id='team_pitching')
    pitching_result = generic_table_analyze(table_pitching)
    # appearances
    appearances_table = content.find('table', id='appearances')
    appearances_result = generic_table_analyze(appearances_table)
    # coaches
    coaches_table = content.find('table', id='coaches')
    coaches_result = generic_table_analyze(coaches_table)
    # standard fielding
    stand_fi_table = content.find('table', id='standard_fielding')
    stand_fi_result = generic_table_analyze(stand_fi_table)
    # players_value_batting
    pv_batting_table = content.find('table', id='players_value_batting')
    pv_batting_result = generic_table_analyze(pv_batting_table)
    #id="players_value_pitching"
    pv_pitching_table = content.find('table', id='players_value_pitching')
    pv_pitching_result = generic_table_analyze(pv_pitching_table)
    
    return {"time_line":time_line_result, 'top': top['df'], 'team_batting': batting_result,
           'team_pitching': pitching_result, 'appearances':appearances_result,
           'coaches':coaches_result, 'stand_fi': stand_fi_result, 'pv_batting': pv_batting_result,
           'pv_pitching': pv_pitching_result}
    

def all_years(content):
    all_time = content.find('div', id='all_franchise_years')
    table = all_time.find('table')
    
    head = table.thead
    column_names = [th.text for th in head.find_all('th')]
    column_exp_stat = [th.attrs['data-stat'] if 'data-stat' in th.attrs else "" for th in head.find_all('th')]
    column_exp_tip = [th.attrs['data-tip'] if 'data-tip' in th.attrs else "" for th in head.find_all('th')]
    df_exp = pd.DataFrame([column_exp_stat, column_exp_tip], columns=column_names)
    
    # year by year datafrome
    data = []
    # details of each year
    data_year = {}
    body = table.tbody
    for tr in body.find_all('tr'):
        header = tr.find('th')
        year = header.text
        
        
        if int(year) < 1980:
            break
        
        team_year_url = header.a['href']
        content = get_parsed_remove_comment(baseball_base_url + team_year_url)
        data_year[year] = each_year(content)
        
        if int(year) == 2020:
            roster_table = content.find('table', id='the40man')
            roster = generic_table_analyze(roster_table)
        
        rest = [td.text for td in tr.find_all('td')]
        
        data.append([year] + rest)
    
    years_df = pd.DataFrame(data, columns=column_names)
    
    return {'years':years_df, 'each_year': data_year, 'exp': df_exp, 'roster_2020': roster}
    

In [208]:
def process_team(content):
    
    small = info(content)
    hist = all_history(content)
    retire = retired(content)
    top = all_time_top(content)
    all_season = all_years(content)
    return small, hist, retire, top, all_season


In [209]:
def process_uniform(content):
    table_grid = content.find(attrs={'data-entry-type':'Uniform Lists'})
    divs = table_grid.find_all('div', attrs={"id" : re.compile('season_leaderboard_*')})
    #print(divs)
    colnames = ['Number', 'Name', 'Year']
    data = []
    for div in divs:
        table = div.find('table')
        
        number = table.caption.text
        #print(number)
        
        for tr in table.find_all('tr'):
            infoo = tr.text
            left_pa = infoo.index('(')
            name = infoo[:left_pa]
            print(name)
            year = infoo[left_pa:]
            data.append([number, name, year])
    
    df = pd.DataFrame(data, columns=colnames)
    return df
        

In [215]:
folder = 'teams'
for rownum in tqdm(range(min(df_all_teams.shape[0], 100))):
    url = df_all_teams.loc[rownum, 'Franchise_url']
    content = get_parsed_remove_comment(baseball_base_url + url)
    small, hist, retire, top, all_season = process_team(content)
    
    url_uniform = baseball_base_url + url + 'uniform-numbers.shtml'
    content = get_parsed_remove_comment(url_uniform)
    uniform = process_uniform(content)
    
    to_save = {'small':small, 'hist_name':hist, 'retire':retire,
              'top':top, 'all_season': all_season, 'uniform': uniform}
    team_name = df_all_teams.loc[rownum, 'Franchise']
    file_name = f'{rownum:03}_' + team_name + '.pkl'
    path_to_save = os.path.join(folder, file_name)
    
    
    with open(path_to_save, 'wb') as f:
        pickle.dump(to_save, f)
    

  3%|██▊                                                                                | 1/30 [00:17<08:37, 17.83s/it]

1
 Jarrod Dyson 
 Michael Bourn 
 Didi Gregorius 
 Cody Ransom 
 Bobby Crosby 
 Orlando Hudson 
 Andy Green 
2
 Starling Marte 
 Domingo Leyba 
 Jeff Mathis 
 Jean Segura 
 Aaron Hill 
 Kelly Johnson 
 Felipe Lopez 
 Chris Burke 
 Damion Easley 
 Roberto Alomar 
 Jerry Gil 
 Juan Sosa 
 Hanley Frias 
3
 Daniel Descalso 
 Tony Abreu 
 Carlos Baerga 
 Danny Klassen 
4
 Ketel Marte 
 Mike Freeman 
 Jamie Romak 
 Cliff Pennington 
 Collin Cowgill 
 Melvin Mora 
 Rusty Ryal 
 David Eckstein 
 Emilio Bonifacio 
 Donnie Sadler 
 Craig Counsell 
 Quinton McCracken 
 Dante Powell 
5
 Eduardo Escobar 
 Alex Avila 
 Gregor Blanco 
 Rickie Weeks 
 Ender Inciarte 
 Cole Gillespie 
 Robby Hammock 
 Koyie Hill 
 Scott Hairston 
 Tony Womack 
 Andy Stankiewicz 
6
 David Peralta 
 Adam Eaton 
 Stephen Drew 
 Quinton McCracken 
 Brent Mayne 
 Ryan Christenson 
 Jason Conti 
 Andy Fox 
7
 Welington Castillo 
 Cody Ross 
 Robby Hammock 
 Carlos Quentin 
 Danny Klassen 
8
 Mike Leake 
 Chris Iannetta 
 Tuf

1
 Ozzie Albies 
 Erick Aybar 
 Alberto Callaspo 
 Emilio Bonifacio 
 Tyler Pastornicky 
 Jordan Schafer 
 Jack Wilson 
 Gregor Blanco 
 Brent Lillibridge 
 Martin Prado 
 Tony Pena 
 Rafael Furcal 
 Otis Nixon 
 Keith Lockhart 
 Luis Polonia 
 Oddibe McDowell 
 Albert Hall 
 Jerry Royster 
 Del Crandall 
 Ray Mueller 
 Tommy Holmes 
 Sibby Sisti 
 Eddie Miller 
 Bama Rowell 
 Gene Moore 
 Rabbit Maranville 
2
 Micah Johnson 
 Drew Stubbs 
 Dansby Swanson 
 Michael Bourn 
 Juan Uribe 
 Melvin Upton Jr. 
 Jeff Baker 
 Jack Wilson 
 Alex Gonzalez 
 Kelly Johnson 
 Jesse Garcia 
 Rico Brogna 
 Rey Sanchez 
 Mark DeRosa 
 Rafael Belliard 
 Albert Hall 
 Barry Bonnell 
 Jim Breazeale 
 Darrell Evans 
 Marty Martinez 
 Frank Bolling 
 Harry Hanebrink 
 Billy Klaus 
 Gene Mauch 
 Al Dark 
 Tommy Neill 
 Johnny Barrett 
 Frank Drews 
 Ben Geraghty 
 Dee Phillips 
 Eddie Joost 
 Sibby Sisti 
 Buddy Hassett 
 Otto Huber 
 Hank Majeski 
 Gil English 
 Billy Urbanski 
 Red Worthington 
3
 Dale Mur

  7%|█████▌                                                                             | 2/30 [00:50<10:24, 22.32s/it]

1/8
 Eddie Gaedel 
1
 Richie Martin 
 Tim Beckham 
 Michael Bourn 
 Everth Cabrera 
 Jemile Weeks 
 Alexi Casilla 
 Brian Roberts 
 Jesse Garcia 
 P.J. Forbes 
 Jerome Walton 
 Tony Tarasco 
 Tommy Shields 
 Juan Bell 
 Phil Bradley 
 Jeff Stone 
 Rex Hudler 
 Al Bumbry 
 Chico Fernandez 
 Bob Saverine 
 Marv Breeding 
 Bob Boyd 
 Chico García 
 Willy Miranda 
 Joe DeMaestri 
 Paul Lehner 
 Billy DeMars 
 John Sullivan 
 Sam Dente 
 Len Schulte 
 Floyd Baker 
 Don Heffner 
 Bill Knickerbocker 
 Lyn Lary 
 Alan Strange 
 Jim Levey 
2
 Dilson Herrera 
 Jonathan Villar 
 Danny Valencia 
 J.J. Hardy 
 Julio Lugo 
 Ryan Freel 
 Luis Montanez 
 Juan Castro 
 Luis Hernandez 
 Bret Barberie 
 Mark McLemore 
 Bob Melvin 
 Alan Wiggins 
 Bobby Bonner 
 Billy Smith 
 Jim Northrup 
 Rich Coggins 
 Jerry DaVanon 
 Bobby Floyd 
 Mickey McGuire 
 Bob Johnson 
 Bob Saverine 
 Johnny Temple 
 Clint Courtney 
 Al Pilarcik 
 Joe Frazier 
 Fred Marsh 
 Jim Brideweser 
 Neil Berry 
 Frank Kellert 
 Snuffy 

...

0
 Junior Ortiz 
 Oddibe McDowell 
 Al Oliver 
1
 Elvis Andrus 
 Desi Relaford 
 Dave Martinez 
 Domingo Cedeno 
 Kurt Stillwell 
 Jeff Frye 
 Gary Green 
 Scott Fletcher 
 Bill Stein 
 Bump Wills 
 Dave Nelson 
 Tim Cullen 
 Frank Coggins 
 Don Blasingame 
 Marv Breeding 
 Tom Brown 
 Jack Jenkins 
 Willie Tasby 
2
 Jeff Mathis 
 Hanser Alberto 
 Leonys Martín 
 Jurickson Profar 
 Taylor Teagarden 
 Adam Melhuse 
 Drew Meyer 
 Eric Young Sr. 
 Andres Torres 
 Ramon Nivar 
 Carl Everett 
 Michael Young 
 Jason McDonald 
 Damon Buford 
 Otis Nixon 
 Manuel Lee 
 Alan Bannister 
 Mike Richardt 
 Sandy Alomar 
 Joe Lovitto 
 Duke Sims 
 Jim Mason 
 Lenny Randle 
 Don Wert 
 Aurelio Rodriguez 
 Ken McMullen 
 John Kennedy 
 Barry Shetrone 
 Ed Brinkman 
 R C Stevens 
3
 Delino DeShields 
 Carlos Corporan 
 Luis Sardinas 
 Leury Garcia 
 Luis Hernandez 
 Andres Blanco 
 Joaquin Arias 
 Jerry Hairston 
 Alex Rodriguez 
 Chad Curtis 
 Mark McLemore 
 Billy Ripken 
 John Russell 
 Jose Hernand

 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [15:29<00:30, 30.82s/it]

0
 Taijuan Walker 
 Al Oliver 
00
 Taijuan Walker 
 Cliff Johnson 
1
 Shun Yamaguchi 
 Alen Hanson 
 Aledmys Díaz 
 Rafael Lopez 
 Andy Burns 
 Jonathan Diaz 
 Emilio Bonifacio 
 Darin Mastroianni 
 Dewayne Wise 
 Joe Inglett 
 Ray Olmedo 
 Bengie Molina 
 Orlando Hudson 
 Tony Fernandez 
 Mickey Morandini 
 Tomas Perez 
 Eddie Zosky 
 Bob Bailor 
2
 Joe Panik 
 Clayton Richard 
 Troy Tulowitzki 
 Kelly Johnson 
 Aaron Hill 
 Dave Berg 
 Cesar Izturis 
 Jacob Brumfield 
 Tom Evans 
 Craig Grebeck 
 Otis Nixon 
 Rob Butler 
 Luis Sojo 
 Manuel Lee 
 Nelson Liriano 
 Fred Manrique 
 Danny Ainge 
 Steve Staggs 
3
 Brandon Drury 
 Gio Urshela 
 Ezequiel Carrera 
 Maicer Izturis 
 Adeiny Hechavarria 
 Dewayne Wise 
 Jarrett Hoffpauir 
 Randy Ruiz 
 Reed Johnson 
 Chris Gomez 
 Orlando Hudson 
 Jeff Frye 
 Norberto Martin 
 Vernon Wells 
 Felipe Crespo 
 Miguel Cairo 
 Mookie Wilson 
 Wayne Nordhagen 
 Leon Roberts 
 Brian Milner 
 Steve Bowling 
4
 Kyle Drabek 
 Shannon Stewart 
 Jorge Vela

0
 Al Oliver 
00
 Brandon Watson 
1
 Wilmer Difo 
 Steve Lombardozzi 
 Nyjer Morgan 
 Willie Harris 
 Tyrell Godwin 
 Josh Labandeira 
 Jose Macias 
 Terry Jones 
 Yamil Benitez 
 Todd Haney 
 Dave Martinez 
 U L Washington 
 Tony Scott 
 Wallace Johnson 
 Willie Davis 
 Gary Sutherland 
2
 Adam Eaton 
 Denard Span 
 Roger Bernadina 
 Felipe Lopez 
 Damian Jackson 
 Jamey Carroll 
 Carl Everett 
 Maicer Izturis 
 Jose Macias 
 Tomas de la Rosa 
 Orlando Cabrera 
 Tim Spehr 
 Jerry Willard 
 Gil Reyes 
 Vance Law 
 Brad Mills 
 Tony Bernazard 
 Rich Coggins 
 Jim Northrup 
 Jim Lyttle 
 Tim McCarver 
 John Bateman 
3
 Michael A. Taylor 
 Asdrubal Cabrera 
 Eury Perez 
 Wilson Ramos 
 Willy Taveras 
 Jesus Flores 
 Jose Vidro 
 Mike Hubbard 
 Mike Lansing 
 Steve Lyons 
 Matt Stairs 
 Junior Noboa 
 Dave Martinez 
 Herm Winningham 
 Bryan Little 
 Rodney Scott 
 Bob Bailey 
4
 Howie Kendrick 
 Reed Johnson 
 Zach Walters 
 David DeJesus 
 Corey Patterson 
 Pete Orr 
 Ryan Langerhans 
 D'

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [16:01<00:00, 32.04s/it]


For sanity check

In [183]:
all_season['each_year']['2003']['top'].head(5)

,Rank,Name,Stat,Image_src
0,1,Brandon Webb,(5.9 WAR),https://www.baseball-reference.com/req/2020121...
1,2,Curt Schilling,(5.7 WAR),https://www.baseball-reference.com/req/2020121...
2,3,Miguel Batista,(3.7 WAR),https://www.baseball-reference.com/req/2020121...
3,4,Luis Gonzalez,(3.2 WAR),https://www.baseball-reference.com/req/2020121...
4,5,Steve Finley,(3.2 WAR),https://www.baseball-reference.com/req/2020121...


In [214]:
to_save['retire']['df']

,Name,Number
0,Luis Gonzalez,20
1,Randy Johnson,51
2,Jackie Robinson (league-wide),42
